# Making the baseline model and getting things to work

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms
from torch import nn, optim
import matplotlib.pyplot as plt
import os
import rasterio
import pickle
from dotenv import load_dotenv

load_dotenv()

device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))

print(f"Training on {device}.")

### Import data

In [2]:
def import_training_imgs(train_folder):
    x_train= []

    for file in os.listdir(train_folder):
        if file.endswith('.tif'):
            file_path = os.path.join(train_folder, file)
            with rasterio.open(file_path) as src:
                image_np = src.read()  
                image_tensor = torch.from_numpy(image_np).float()
                x_train.append(image_tensor)
    return x_train
x_train = import_training_imgs(os.getenv('IMAGES_PATH'))

In [ ]:
len(x_train)

In [5]:
def import_training_labels(train_folder):
    y_train = []

    for file in os.listdir(train_folder):
        if file.endswith('.tif'):
            file_path = os.path.join(train_folder, file)
            with rasterio.open(file_path) as src:
                image_np = src.read()
                image_tensor = torch.from_numpy(image_np)
                y_train.append(image_tensor)
    return y_train

In [18]:
y_train = import_training_labels(os.getenv('MASKED_IMAGES_PATH'))

In [ ]:
len(y_train)

In [20]:

x_train_tensor = torch.stack(x_train, dim=0)  # Shape: [num_samples, 12, 1024, 1024]
y_train_tensor = torch.stack(y_train, dim=0).squeeze(1).long()   # Shape: [num_samples, 1, 1024, 1024]


In [21]:
x_train_tensor = torch.nan_to_num(x_train_tensor, nan=0.0)

In [ ]:
means = x_train_tensor.mean(dim=(0, 2, 3))
stds = x_train_tensor.std(dim=(0, 2, 3))
print("Means:", means)
print("Stds:", stds)

eps = 1e-7
stds_fixed = stds + eps

normalizer_pipe = transforms.Normalize(means, stds_fixed)

preprocessor = transforms.Compose([
    normalizer_pipe
])

x_train_tensor = [preprocessor(img) for img in x_train_tensor]
x_train_tensor = torch.stack(x_train_tensor, dim=0)


In [ ]:
x_train_tensor.shape

In [ ]:
y_train_tensor.shape

In [ ]:
type(x_train_tensor)

In [26]:
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)


In [ ]:
x_train_tensor

In [28]:
batch_size = 10

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

### Training function

In [35]:
from train.train import train

###  A simple Convolutional Network

In [ ]:
from models.simple_convnet import SimpleConvNet

In [36]:
model = SimpleConvNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
n_epochs = 10

losses_train = train(n_epochs, optimizer, model, loss_fn, train_loader)
